In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sweetviz as sv
import missingno as msno
import time 
import sys
import os
import csv
from bs4 import BeautifulSoup
import requests
import re

from sklearn.compose import make_column_selector, ColumnTransformer, make_column_transformer
# sklearn.compose: The sklearn.compose module is a submodule of the sklearn library for machine learning in Python. It provides functions for creating complex preprocessing and modeling pipelines.
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PolynomialFeatures,RobustScaler
#sklearn.preprocessing: The sklearn.preprocessing module is a submodule of the sklearn library that provides functions for preprocessing data, such as scaling and normalizing features, imputing missing values, and encoding categorical variables.
from sklearn.linear_model import Ridge,LinearRegression,Lasso, ElasticNet
# sklearn.linear_model: The sklearn.linear_model module is a submodule of the sklearn library that provides functions for fitting linear models for regression and classification.
from sklearn.pipeline import make_pipeline
# sklearn.pipeline: The sklearn.pipeline module is a submodule of the sklearn library that provides functions for creating and working with pipelines of transformers and models.
from sklearn.model_selection import train_test_split,GridSearchCV,learning_curve, RandomizedSearchCV, cross_val_score, KFold
# sklearn.model_selection: The sklearn.model_selection module is a submodule of the sklearn library that provides functions for splitting data into training and test sets, evaluating models using cross-validation, and hyperparameter tuning.
from sklearn.dummy import DummyRegressor
# sklearn.dummy: The sklearn.dummy module is a submodule of the sklearn library that provides simple dummy models for regression and classification.


from sklearn.impute import SimpleImputer
import numpy as np

from sklearn.ensemble import RandomForestClassifier
import pickle


In [2]:
try:
    df = pd.read_csv('/home/apprenant/Documents/archive/SBAnational.csv')
except:
    df = pd.read_csv('C:/Users/emada/Downloads/loan_project (1)/SBAnational.csv')

df['ApprovalFY'] = df['ApprovalFY'].replace('A', '', regex=True).astype(int)

/tmp/ipykernel_23275/1292895530.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/apprenant/Documents/archive/SBAnational.csv')


In [6]:
# create a list of all numeric columns
columns_to_transform_to_int = ["DisbursementGross","BalanceGross","ChgOffPrinGr","GrAppv","SBA_Appv"]


for col in columns_to_transform_to_int:
    df[col] = df[col].str.replace("$", "")
    df[col] = df[col].str.replace(",", "")
    df[col] = df[col].astype(float)
    

df['Term'] = df['Term'].astype(int)

df['LowDoc'] = df['LowDoc'].replace({'0': 'N'})
df = df[df['LowDoc'].isin(['N','Y'])]
df['RevLineCr'] = df['RevLineCr'].replace({'0': 'N', 'T':'Y'})
df = df[df['RevLineCr'].isin(['N','Y'])]

dictionnaire =  {"11":"Agriculture, forestry, fishing and hunting","72":"Accommodation and food services","21":"Mining, quarrying, and oil and gas extraction","22":
"Utilities","23":"Construction","31":"Manufacturing","32":"Manufacturing","33":"Manufacturing","42":"Wholesale trade","44":"Retail trade","45":"Retail trade","48":" Transportation and warehousing", "49":"Transportation and warehousing", "51":"Information","52":"Finance and insurance", "53":"Real estate and rental and leasing","54":"Professional, scientific, and technical services","55":"Management of companies and enterprises","56":"Administrative and support and waste management and remediation services","61":"Educational services","62":"Health care and social assistance","71":"Arts, entertainment, and recreation","81":"Other services (except public administration)", "92": "Public administration", "0" : "Other"}

df['NAICS'] = df['NAICS'].astype(str).str[:2]

df["NAICS"] = df["NAICS"].map(dictionnaire)

cols_to_drop = ['LoanNr_ChkDgt', 'Name', 'City', 'Zip', 'Bank', 'BankState', 'ApprovalDate', 'RetainedJob','ChgOffDate', 'DisbursementDate', 'DisbursementGross', 'BalanceGross', 'SBA_Appv', 'ChgOffPrinGr']
df_cleaned = df.copy()
df_cleaned.drop(columns=cols_to_drop, inplace=True)

df_cleaned.dropna(subset=['MIS_Status'], inplace=True)


/tmp/ipykernel_23275/2495417656.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace("$", "")


In [9]:
df['CreateJob'].value_counts()

0       623611
1        62519
2        57106
3        28431
4        20232
         ...  
3100         1
183          1
226          1
210          1
860          1
Name: CreateJob, Length: 245, dtype: int64

In [4]:
y = df_cleaned.MIS_Status
X = df_cleaned.drop(columns=['MIS_Status'])

NameError: name 'df_cleaned' is not defined

In [5]:
def make_pipeline_to_ML(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8, random_state=42)
    numerical_features = make_column_selector(dtype_include=np.number)
    categorical_features = make_column_selector(dtype_exclude= np.number)
    numerical_pipeline = make_pipeline(SimpleImputer(strategy='mean'),StandardScaler(with_mean=False))
    categorical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder(handle_unknown='ignore'))
    preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                    (categorical_pipeline, categorical_features)
                                    )
    return preprocessor, X_train, X_test, y_train, y_test


# Create the pipeline
preprocessor, X_train, X_test, y_train, y_test = make_pipeline_to_ML(X,y)

NameError: name 'X' is not defined

# XGBClassifier
!['XGD'](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRD7Jd4XJNHb0wB_i4V7NFA_qrc6LTEuxPhgw&usqp=CAU)

In [6]:
from sklearn.preprocessing import LabelEncoder

# initialize the label encoder
le = LabelEncoder()

# fit and transform the y labels
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


In [10]:
from xgboost import XGBClassifier
# Add the XGBClassifier to the pipeline
clf_xgb = make_pipeline(preprocessor, XGBClassifier(random_state=42, 
                                                max_depth = 20, 
                                                n_estimators = 1000,
                                                min_child_weight = 4800,
                                                subsample = 0.8,
                                                colsample_bytree = 0.8,
                                                scale_pos_weight=1.375,
                                                objective='binary:logistic',
                                                learning_rate=0.3 #par default 
                                                n_jobs=-1))

# Fit the pipeline on the training data
clf_xgb.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler(with_mean=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f747c147f70>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_freque...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=20, max_leaves=None,
                               min_child_weight=4800, missing=nan,
                               monotone_constraints=None, n_estimators=1000,
                               n_jobs=-1, num_parallel_tree=None,
                               predictor=None, random_state=42, ...))])

In [11]:
# Get the accuracy score on the test data
accuracy = clf_xgb.score(X_test, y_test)

# Print the accuracy score
print("Accuracy: {:.2f}%".format(accuracy * 100))


Accuracy: 93.28%


In [15]:
clf_xgb = make_pipeline(preprocessor, XGBClassifier(random_state=42, 
                                                max_depth = 20, 
                                                n_estimators = 1000,
                                                min_child_weight = 4800,
                                                subsample = 0.8,
                                                colsample_bytree = 0.8,
                                                scale_pos_weight=1.375,
                                                objective='binary:logistic',
                                                learning_rate=0.2,
                                                n_jobs=-1))

# Fit the pipeline on the training data
clf_xgb.fit(X_train, y_train)

# Get the accuracy score on the test data
accuracy = clf_xgb.score(X_test, y_test)

# Print the accuracy score
print("Accuracy: {:.2f}%".format(accuracy * 100))


Accuracy: 93.25%
